In [1]:
from __future__ import print_function
import numpy as np
import warnings

from keras.models import Model 
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import GlobalMaxPool2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
#from keras.utils import layers
from keras.utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import obtain_input_shape
#from keras.engine.topology import get_source_inputs

In [2]:
def VGGUpdated(input_tensor=None, classes=26):
    img_rows, img_cols = 224, 224 # by default size is 224, 224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)
    img_input = Input(shape=img_dim)

    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPool2D((2,2), strides=(2,2), name='block1_pool')(x)

    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPool2D((2,2), strides=(2,2), name='block2_pool')(x)

    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPool2D((2,2), strides=(2,2), name='block3_pool')(x)

    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPool2D((2,2), strides=(2,2), name='block4_pool')(x)

    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3,3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPool2D((2,2), strides=(2,2), name='block5_pool')(x)

    # classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    model = Model(inputs=img_input, outputs=x, name='VGG')
    return model

In [3]:
model = VGGUpdated(classes=26)

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('../../02_data_crop_abc/train')
classes = os.listdir('../../02_data_crop_abc/train')
print("Types of classes found: ", len(dataset_path))

Types of classes found:  26


In [7]:
imagePath_label_pair = []
for item in classes:
    all_letters = os.listdir('../../02_data_crop_abc/train' + '/' + item)
    for letter in all_letters:
        imagePath_label_pair.append((item, str('../../02_data_crop_abc/train' + '/' + item) + '/' + letter))

In [7]:

""" 
letter_df = pd.DataFrame(data=letters, columns=['letter', 'image'])

# Definieren der Batch-Größe
batch_size = 64

# Aufteilen des DataFrames in Batches
batches = [letter_df[i:i+batch_size] for i in range(0, len(letter_df), batch_size)]

# Iteration über die Batches
for batch in batches:
    # Hier können Sie Ihre Verarbeitungsschritte für jeden Batch durchführen
    print(batch)
"""

" \nletter_df = pd.DataFrame(data=letters, columns=['letter', 'image'])\n\n# Definieren der Batch-Größe\nbatch_size = 64\n\n# Aufteilen des DataFrames in Batches\nbatches = [letter_df[i:i+batch_size] for i in range(0, len(letter_df), batch_size)]\n\n# Iteration über die Batches\nfor batch in batches:\n    # Hier können Sie Ihre Verarbeitungsschritte für jeden Batch durchführen\n    print(batch)\n"

In [8]:
# build a dataframe
letter_imagePath = pd.DataFrame(data=imagePath_label_pair, columns=['letter', 'image'])
print(letter_imagePath.head())

  letter                                       image
0      a  ../../02_data_crop_abc/train/a/a_00001.jpg
1      a  ../../02_data_crop_abc/train/a/a_00002.jpg
2      a  ../../02_data_crop_abc/train/a/a_00003.jpg
3      a  ../../02_data_crop_abc/train/a/a_00004.jpg
4      a  ../../02_data_crop_abc/train/a/a_00005.jpg


In [9]:
print("Total number: ", len(letter_imagePath))
letter_count = letter_imagePath['letter'].value_counts()
print('images in each class: ')
print(letter_count)

Total number:  10368
images in each class: 
letter
a    494
e    449
b    444
i    440
l    430
n    428
k    426
v    425
o    417
t    409
s    408
u    407
w    402
m    398
r    398
p    396
h    395
d    393
g    392
f    390
c    389
q    386
x    381
y    373
j    321
z    177
Name: count, dtype: int64


In [25]:
""" 
import cv2
path = '../../02_data_crop_abc/train/'

img_size = 224

images = []
labels = []

batch_size = 64
for i in class_types:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]
    for j in range(0, len(filenames), batch_size):
        batch_images = []
        batch_labels = []
        for f in filenames[j:j+batch_size]:
            img = cv2.imread(data_path + '/' + f)
            img = cv2.resize(img, (img_size, img_size))
            batch_images.append(img)
            batch_labels.append(i)
        batch_images = np.array(batch_images)
        batch_images = batch_images.astype('float32') / 255.0
        images.append(batch_images)
        labels.extend(batch_labels)
"""

In [10]:

import cv2
path = '../../02_data_crop_abc/train/'

img_size = 224

images = []
labels = []

for i in classes:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(i)


In [11]:

images = np.array(images)
images = images.astype('float32') / 255.0
images.shape


(10368, 224, 224, 3)

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = letter_imagePath['letter'].values

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

[ 0  0  0 ... 25 25 25]


In [13]:
y = y.reshape(-1,1)
onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(y)
Y.shape

(10368, 26)

In [14]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(9849, 224, 224, 3)
(9849, 26)
(519, 224, 224, 3)
(519, 26)


In [15]:
def data_generator(images, labels, batch_size):
    num_samples = len(images)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_images = images[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            yield batch_images, batch_labels

batch_size = 64
train_generator = data_generator(train_x, train_y, batch_size)
test_generator = data_generator(test_x, test_y, batch_size)

model.fit(train_generator, epochs=5, steps_per_epoch=len(train_x)//batch_size)

Epoch 1/5


TypeError: Failed to convert elements of SparseTensor(indices=Tensor("data_1:0", shape=(None, 2), dtype=int64), values=Tensor("compile_loss/categorical_crossentropy/Cast:0", shape=(None,), dtype=float32), dense_shape=Tensor("data_3:0", shape=(2,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.

In [ ]:
#model.fit(train_x, train_y, epochs=5, batch_size=64)

MemoryError: Unable to allocate 5.52 GiB for an array with shape (9849, 224, 224, 3) and data type float32

In [ ]:
preds = model.evaluate(test_x, test_y)
print("Loss= " + str(preds[0]))
print("Test acc= " + str(preds[1]))

In [ ]:
import matplotlib.pyplot as plt

img_path = 'test.jpg'
img = image.load_img(img_path, target_size=(224,224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print('Input image shape: ', x.shape)

In [ ]:
my_image = plt.imread(img_path)
plt.imshow(my_image)

In [ ]:
print(model.predict(x))